Realizamos la conexion al telefono en el puerto 9999 corriendo el servidor en el telefono SL4A

In [8]:
!adb forward tcp:9999 tcp:9999

error: device '(null)' not found
error: device '(null)' not found


In [9]:
!adb devices

List of devices attached



In [5]:
%matplotlib inline
import time 
import android
import matplotlib.pyplot as plt
import cv2
import numpy as np
import sys, os,re
import pprint



def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err
 
def compare_images(imageA, imageB, title):
	# compute the mean squared error and structural similarity
	# index for the images
	m = mse(imageA, imageB)
	print  str(m)
	# s = ssim(imageA, imageB)
 
	# setup the figure
	fig = plt.figure(title)
	#plt.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))
 
	# show first image
	ax = fig.add_subplot(1, 2, 1)
	plt.imshow(imageA, cmap = plt.cm.gray)
	plt.axis("off")
 
	# show the second image
	ax = fig.add_subplot(1, 2, 2)
	plt.imshow(imageB, cmap = plt.cm.gray)
	plt.axis("off")
 
	# show the images
	plt.show()
	return m



def revisar():
	droide=android.Android()
	counter=1
	delay=5
	numofShots = 5
	delayBetweenShots=8
	couterDecteccion=1
	droide.ttsSpeak('Esperando alerta')
	time.sleep(int(1))
	mensaje=''
	bandera=0    
	while bandera==0: 
		droide.ttsSpeak('Esperando mensaje')
		msgs = droide.smsGetMessages(False) 
		total=droide.smsGetMessageCount(False).result
		contador=0
		while contador<total:
			if msgs.result[contador]['address'] is not None:
				if msgs.result[contador]['address']=='2851087970' or msgs.result[contador]['address']=='2281248059' or msgs.result[contador]['address']=='2299848788':
					mensaje= msgs.result[contador]['body']
					break
					bandera=1
			contador+=1
	while mensaje!='' : 
		droide.ttsSpeak('Tomando foto 1')
		droide.cameraCapturePicture('/sdcard/DCIM/CAMERA/programa/foto_'+ str(couterDecteccion)+ '_.jpg',1,1)
		os.system('adb pull /sdcard/DCIM/CAMERA/programa/foto_'+ str(couterDecteccion)+ '_.jpg')
		pic = os.path.join('.','foto_'+  str(couterDecteccion)+ "_.jpg")
		img1 = cv2.imread(pic)
		couterDecteccion +=1
		time.sleep(int(delayBetweenShots))
		#droide.ttsSpeak('Tomando foto 2')
		droide.cameraCapturePicture('/sdcard/DCIM/CAMERA/programa/foto_'+ str(couterDecteccion)+ '_.jpg',1,1)
		os.system('adb pull /sdcard/DCIM/CAMERA/programa/foto_'+ str(couterDecteccion)+ '_.jpg')
		pic = os.path.join('.','foto_'+  str(couterDecteccion)+ "_.jpg")
		couterDecteccion +=1
		img2 =  cv2.imread(pic)
		plt.imshow(img1)
		plt.imshow(img2)
		i= img1.copy()
		i2=img2.copy()
		diferente = compare_images(i,i2,'pruebacompare')
		counter=0
		if diferente >1000:
			droide.ttsSpeak('Imagen con cambio')
			while counter <= int(numofShots):
				droide.ttsSpeak('Tomando foto carrete '+ str(counter))
				droide.cameraCapturePicture('/sdcard/DCIM/CAMERA/programa/fotoC_'+ str(counter)+ '_.jpg',1,1)
				os.system('adb pull /sdcard/DCIM/CAMERA/programa/fotoC_'+ str(counter)+ '_.jpg')
				pic = os.path.join('.','fotoC_'+  str(counter)+ "_.jpg")
				counter +=1
				img2 =  cv2.imread(pic)
				plt.imshow(img1)
				plt.imshow(img2)
				i= img1.copy()
				i2=img2.copy()
				diferente =compare_images(i,i2,'pruebacompare')
				if diferente <1000:
					droide.ttsSpeak('Imagen parecida')
					break
				time.sleep(int(1))
		##if response['which']=='negative':
		droide.dialogShow()
		response = droide.dialogGetResponse().result
	print 'Hecho sin errores ...'
	del droide    
    
    
revisar()

KeyboardInterrupt: 